In [32]:
i=200

In [33]:
import requests
import re
import pandas as pd
from string import digits
from bs4 import BeautifulSoup

In [34]:
url = "http://www.banki.ru/services/responses/list/"
response = requests.get(url)
i_max = 300
soup = BeautifulSoup(response.text, "html5lib")

In [35]:
reviews = soup.find_all("article", attrs={"class": "responses__item"})
df = pd.DataFrame(columns = ('date', 'bank', 'title', 'rating', 'text'), index = [0])

в цикле переходим между страницами, на каждой странице проверяем каждый отзыв, при возможности считываем полный отзыв по ссылке. оцищаем от пунктуации и лишних пробелов. пишем все в DataFrame

In [36]:
while (response and i < i_max) :
    for review in reviews:
    # дата
        date = review.find("time", "display-inline-block").text

    # название банка
        bank_name = review.find_all("a", attrs={"class": "responses__item__logo lazy-load"})[0].get("title")
        bank_name = re.sub(' +',' ', bank_name)
        bank_name = re.sub('[A-Za-z0-9]+\s+', '', bank_name)

    # заголовок
        title = review.find_all("a", attrs={"class": "font-size-large font-bold"})[0].text
        title = re.sub(' +',' ', title)
        title = re.sub('[A-Za-z0-9]+\s+', '', title)    

    # рейтинг
        rating = review.find("div", attrs={"class": "responses__item__rating"}).find("meta")
        if rating: rating = rating.get("content")
        else: rating = None

    # полный текст отзыва
        link = review.find("div", attrs={"responses__item__message"})
        if link is not None:
            if (link.find("a", href=True) is not None):
                url1 = "http://www.banki.ru" + link.find("a", href=True)['href']
                text_page = requests.get(url1)
                soup1 = BeautifulSoup(text_page.text, "html5lib")
                text = soup1.find("div", "article-text response-page__text markup-inside-small markup-inside-small--bullet")
                if (text is not None): text = text.text 
                else: text = review.find("div", "responses__item__message").text
            else: text = review.find("div", "responses__item__message").text 
        else: text = review.find("div", "responses__item__message").text
        text = re.sub(' +',' ', text)
        text = re.sub('[A-Za-z0-9]+\s+', '', text)
        remove_digits = str.maketrans('', '', digits)
        text = text.translate(remove_digits).lstrip().rstrip()

    # запись в DataFrame
        df = df.append({
            'date': date,
            'bank': bank_name,
            'title': title,
            'rating': rating,
            'text': text[:150]
        }, ignore_index=True)
    
    # листаем по страницам
    i+=1
    url2 = url+"?page="+str(i)
    response = requests.get(url2)
    soup = BeautifulSoup(response.text, "html5lib")
    reviews = soup.find_all("article", attrs={"class": "responses__item"})
    print (url2)

http://www.banki.ru/services/responses/list/?page=201
http://www.banki.ru/services/responses/list/?page=202
http://www.banki.ru/services/responses/list/?page=203
http://www.banki.ru/services/responses/list/?page=204
http://www.banki.ru/services/responses/list/?page=205
http://www.banki.ru/services/responses/list/?page=206
http://www.banki.ru/services/responses/list/?page=207
http://www.banki.ru/services/responses/list/?page=208
http://www.banki.ru/services/responses/list/?page=209
http://www.banki.ru/services/responses/list/?page=210
http://www.banki.ru/services/responses/list/?page=211
http://www.banki.ru/services/responses/list/?page=212
http://www.banki.ru/services/responses/list/?page=213
http://www.banki.ru/services/responses/list/?page=214
http://www.banki.ru/services/responses/list/?page=215
http://www.banki.ru/services/responses/list/?page=216
http://www.banki.ru/services/responses/list/?page=217
http://www.banki.ru/services/responses/list/?page=218
http://www.banki.ru/services

In [37]:
df[1:].to_csv('data2.csv', encoding = 'utf8')

In [30]:
df

,date,bank,title,rating,text
0,NaN,NaN,NaN,NaN,NaN
1,11.03.2018 16:40,Совкомбанк,После окончания вклада используют личные данны...,1,..с телефона +-в -местного челябинского времен...
2,11.03.2018 16:37,Тинькофф Банк,Отличный банк!!!,5,Пользуюсь по рекомендации друга уже года ! Име...
3,11.03.2018 16:28,Тинькофф Банк,Бардак в Тинкофф банке,1,Обслуживаюсь в банке достаточное длительное вр...
4,11.03.2018 16:21,Сбербанк России,"Навязывают кредитные карты, обман",1,Примерно в -годах пользовался дебетовой картой...
5,11.03.2018 16:00,Совкомбанк,Закрытие вклада без заявления!,1,Сегодня заканчивается срок вклада по договору....
6,11.03.2018 15:50,Тинькофф Банк,Требую решение проблемы,None,Здравствуйте!!!!! Мне подключили овердрафт на ...
7,11.03.2018 15:50,Сетелем Банк,Штраф за не продление полиса каско,1,"..года я, Шишков Николай Александрович, заключ..."
8,11.03.2018 15:35,Тинькофф Банк,Всё путём!,5,Давно собирался написать отзыв. Пользуюсь кред...
9,11.03.2018 15:31,ВТБ,поломали систему поддержки от бм,None,"на сайте bm.была хорошая система поддержки, чт..."
